In [90]:
import pandas as pd

def get_dataframe():
    df = pd.read_csv('data.csv')
    df = df.rename(columns={'Country Code': 'Country_Code'})

    def convert_to_num(value):
        if isinstance(value, str):
            if 'B' in value:
                return float(value.replace('$', '').replace('B', '').strip()) * 1e9  # Billion
            elif 'M' in value:
                return float(value.replace('$', '').replace('M', '').strip()) * 1e6  # Million
        return value 

    cc = df.filter(like='Stock Price').columns
    df[cc] = df[cc].map(convert_to_num)
    cc = df.filter(like='Market Share').columns
    df[cc] = df[cc].map(convert_to_num)
    cc = df.filter(like='Revenue').columns
    df[cc] = df[cc].map(convert_to_num)
    cc = df.filter(like='Expense').columns
    df[cc] = df[cc].map(convert_to_num)
    
    print((df.loc[5,'Stock Price (2024)']))
    return df
df = get_dataframe()

2650000000.0


In [91]:
df.head()

,SL No,Company,Country,Country_Code,Market Cap,Diversity,Stock Price (2015),Stock Price (2016),Stock Price (2017),Stock Price (2018),...,Market share (2015),Market share (2016),Market share (2017),Market share (2018),Market share (2019),Market share (2020),Market share (2021),Market share (2022),Market share (2023),Market share (2024)
0,1,Zooxo,Ukraine,UAH,$2.34B,43.5,6.361900e+08,3.617000e+10,NaN,1.060000e+09,...,28.24,42.01,73.14,90.64,56.80,48.79,35.40,65.88,38.85,77.69
1,2,Gabtype,Brazil,BRL,$30.67B,53.4,1.330000e+09,1.273600e+11,1970000.0,1.920100e+08,...,75.06,71.20,11.70,37.98,67.95,40.41,65.85,55.78,96.02,4.96
2,3,Quinu,China,CNY,$26.32M,83.0,9.390200e+08,1.170000e+09,54950000.0,1.041100e+08,...,89.76,5.96,14.47,34.17,47.13,98.81,21.61,24.12,39.43,95.50
3,4,Youbridge,Colombia,COP,$695.78M,89.2,5.989100e+08,3.635000e+07,NaN,1.370000e+09,...,99.23,20.26,11.82,1.46,54.58,53.21,10.06,98.89,67.63,36.43
4,5,Oyope,Ukraine,UAH,$1.28B,23.3,2.050000e+09,3.999500e+08,637420000.0,1.840000e+09,...,30.98,9.56,85.57,21.37,6.90,91.74,50.53,61.35,76.39,11.98


In [119]:

def findthings(Comp, Country):
    key_row_df = df[(df['Company'] == Comp) & (df['Country_Code'] == Country)]
    
    if key_row_df.empty:
        return 
        
    key_row = key_row_df.iloc[0]
    # print(key_row)
    key_country_code = key_row['Country_Code']
    
    # a - How many companies are there in the same country
    freq = df[df['Country_Code'] == key_country_code].shape[0]
    
    # b - How many companies with greater diversity are in the same country
    moreDiv = df[(df['Country_Code'] == key_country_code) & (df['Diversity'] > key_row['Diversity'])].shape[0]
    
    # c - Inc/Dec in <params> (here we send the actual values, and can find diff array when graphing)
    stock_prices = key_row_df.filter(like='Stock Price').iloc[0]
    market_shares = key_row_df.filter(like='Market share').iloc[0]
    revenues = key_row_df.filter(like='Revenue').iloc[0]
    expenses = key_row_df.filter(like='Expense').iloc[0]

    
    # d - how many companies have greater <params> than this company
    ans = []
    params = ['Stock Price', 'Market share', 'Revenue', 'Expense']
    for param in params:
        latest_param_val = key_row.filter(like=param).iloc[-1]
        
        # domestically
        n_greater_val_dom = df[(df['Country_Code'] == key_country_code) & 
        (df.filter(like=param).iloc[:, -1] > latest_param_val)].shape[0]
        
        # globally
        n_greater_val_glob = df[(df.filter(like=param).iloc[:, -1] > latest_param_val)].shape[0]

        ans.append({'param':param, 'dom_cnt': n_greater_val_dom, 'glob_cnt': n_greater_val_glob})
    # print(ans)


    # lat = key_row.filter(like="Market share").iloc[-1]
    # n_greater_market_share = df[(df['Country_Code'] == key_country_code) & 
    # (df.filter(like='Market share').iloc[:, -1] > lat)]
    # print(lat,"\n", n_greater_market_share[['Market share (2024)']])
    
    return {"a": freq, "b": moreDiv, "c":[stock_prices, market_shares, revenues, expenses],"d":ans}
findthings('Oyope','UAH')

{'a': 13,
 'b': 10,
 'c': [Stock Price (2015)    2050000000.0
  Stock Price (2016)     399950000.0
  Stock Price (2017)     637420000.0
  Stock Price (2018)    1840000000.0
  Stock Price (2019)      11240000.0
  Stock Price (2020)    3380000000.0
  Stock Price (2021)      18560000.0
  Stock Price (2022)    1660000000.0
  Stock Price (2023)     162720000.0
  Stock Price (2024)     252540000.0
  Name: 4, dtype: object,
  Market share (2015)    30.98
  Market share (2016)     9.56
  Market share (2017)    85.57
  Market share (2018)    21.37
  Market share (2019)     6.90
  Market share (2020)    91.74
  Market share (2021)    50.53
  Market share (2022)    61.35
  Market share (2023)    76.39
  Market share (2024)    11.98
  Name: 4, dtype: float64,
  Revenue (2015)    $93589034.46
  Revenue (2016)     $2150706.19
  Revenue (2017)    $18188663.63
  Revenue (2018)    $69999698.77
  Revenue (2019)    $14672498.92
  Revenue (2020)    $24624093.71
  Revenue (2021)    $60783357.79
  Revenue (